In [15]:
import pandas as pd

import numpy as np

from functools import reduce

#import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context = 'notebook', #mostly controls relative sizes of things on plot #The base context is “notebook”, and the other contexts are “paper”, “talk”, and “poster”
        style = 'darkgrid', #dict, None, or one of {darkgrid, whitegrid, dark, white, ticks}
        palette = 'deep', # Should be something that color_palette() can process.
        font_scale = 1, 
        color_codes = False, 
        rc = None)

from scipy import stats

# from IPython.core.interactiveshell import InteractiveShell 
# InteractiveShell.ast_node_interactivity = 'last_expr' #s etting = "all" allows multiple outputs to be displayed for a given input cell. don't use w plotting!

from IPython.display import display

%matplotlib notebook
#%matplotlib inline

pd.__version__, sns.__version__ , np.__version__ #,  matplotlib.__version__

('0.23.3', '0.9.0', '1.14.2')

In [16]:
# some functions to load
def head_with_full_columns(pd_in, row_amount = 5):
    with pd.option_context('display.max_columns', len(pd_in.iloc[0])):
        display(pd_in[:row_amount])
        
def balanced_sample(df_in, total_size, rand_state):
    s0 = df_in[df_in['TARGET']==0].sample(n = total_size//2, random_state = rand_state)
    s1 = df_in[df_in['TARGET']==1].sample(n = total_size//2, random_state = rand_state)
    new_df = pd.concat([s0,s1])
    new_df.sort_index(inplace = True)
    return new_df

In [17]:
cd

/Users/DonBunk


In [18]:
raw_data_path = 'Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/raw_loan_data_from_Kaggle/'

agg_data_path = 'Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/aggregation/TRAIN_aggregation/'

save_path = 'Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/wrangling/TRAINING_DATA_create_final_wrangled_csv/'

#  Wrangle  application_train

In [19]:
application_train_df = pd.read_csv(raw_data_path + 'application_train.csv', index_col = 'SK_ID_CURR')
# AMT_ANNUITY appears in multiple files, so rename more specifically. 
application_train_df.rename(columns = {"AMT_ANNUITY":"AMT_ANNUITY_from_app_train"} , inplace=True) #index = str, 

#application_train_df.sort_index(axis=1, inplace=True)

## null counts feature, social_counts feature, THEN replace categorical NaNs -> 'NA's, combine cat features <1%. creat DAYS_EMPLOYED = 365243 feature, fix DAYS_EMPLOYED

In [20]:
# dropping some features

# separate out certain features 
housing_feats = [f for f in application_train_df.columns if 'AVG' in f or 'MODE' in f or 'MEDI' in f]
doc_feats = [f for f in application_train_df.columns if 'DOCUMENT' in f]
social_circle_feats = [f for f in application_train_df.columns if 'SOCIAL' in f]
amt_req_feats = [f for f in application_train_df.columns if 'AMT_REQ' in f]

# in another notebook (application_less_important_features) I explore these feats and decide what to drop
# this is a bit confusing: I am forming a list of features to drop from groups I made
# and aubtract some features from those groups TO KEEP
# I also add some individual features to drop

groups_of_features_to_drop  = housing_feats + doc_feats + social_circle_feats+ amt_req_feats

feats_to_not_drop = {'EMERGENCYSTATE_MODE', 'APARTMENTS_AVG', 'LANDAREA_AVG', 'TOTALAREA_MODE', # these have a bit of info in them
                        'FLAG_DOCUMENT_3','FLAG_DOCUMENT_6', # these have a bit of info in them
                        'OBS_60_CNT_SOCIAL_CIRCLE',   # there is no info here, but keeping it as representative feature 
                        'AMT_REQ_CREDIT_BUREAU_YEAR'  # there is not a lot of info here, but keeping it as representative feature 
                        }

misc_feats_to_drop = ['FLAG_MOBIL',  # this doesn't have much info
                        'FLAG_CONT_MOBILE', # this doesn't have much info
                        'REGION_RATING_CLIENT_W_CITY' # this is HIGHLY correlated (~.95) with REGION_RATING_CLIENT_
                        ]

feats_to_drop = list( set(groups_of_features_to_drop) - set(feats_to_not_drop)) + misc_feats_to_drop


application_train_df.drop(columns= feats_to_drop, inplace=True)

del housing_feats
del doc_feats
del social_circle_feats
del amt_req_feats
del feats_to_drop

In [22]:
application_train_df.shape

(307511, 49)

In [7]:
# create a null counts feature
null_counts = application_train_df.isnull().sum(axis = 1)
null_counts.rename('NULL_COUNTS', inplace=True);
application_train_df = pd.concat([application_train_df, null_counts], axis = 1)
del null_counts

In [8]:
# create a null counts just for social features. which make have some corr with TARGET
social_circle_feats = [f for f in application_train_df.columns if 'SOCIAL' in f]
social_null_counts = application_train_df[social_circle_feats].isnull().sum(axis = 1)
social_null_counts.rename('SOCIAL_NULL_COUNTS', inplace=True);
application_train_df = pd.concat([application_train_df, social_null_counts], axis = 1)
del social_circle_feats
del social_null_counts

In [9]:
# replace NaNs in object entries for now is pretty safe. 
f_list = list(application_train_df.select_dtypes('object').columns)
application_train_df[f_list] = application_train_df[f_list].fillna(value = 'No/Av')
del f_list

In [10]:
# for categorical feature than have more than one value <1%, condensed the <1% values and define a new condensed feature

for feat in application_train_df.select_dtypes('object').columns:
    percent_srs = application_train_df[feat].value_counts() /len(application_train_df[feat])
    percent_less_than_1 = list(percent_srs[percent_srs <.01].index)
    if len(percent_less_than_1)>1:
        application_train_df[feat + '_condensed'] = application_train_df[feat].apply(lambda x: '<1%' if x in percent_less_than_1 else x)
        application_train_df.drop(columns=[feat], inplace=True)
      
del feat
del percent_srs
del percent_less_than_1

In [11]:
# creat a DAYS_EMPLOYED = 365243 feature, and then regress away those values

def make_days_empl_col(row):
    if row['DAYS_EMPLOYED']== 365243:
        return 'Y'
    else:
        return 'N'
    
from sklearn import linear_model
reg = linear_model.LinearRegression()

# have to make a list of lists, turn that into np.array, then transpose to has correct shape for the single feature(s) working w here. 
no_weird_vals_df = application_train_df[ application_train_df['DAYS_EMPLOYED'] !=365243]
reg.fit(np.array([no_weird_vals_df['DAYS_BIRTH'].values]).T, no_weird_vals_df['DAYS_EMPLOYED'].values)

# create a new binary column while I still have the 365243 values.
application_train_df['DAYS_EMPLOYED_eq_365243'] = application_train_df.apply(make_days_empl_col, axis=1)

# replace ['DAYS_EMPLOYED']==365243 with lin reg vals from above

def fix_DAYS_EMPLOYED(row):
    # fill in the weird values
    if row['DAYS_EMPLOYED']== 365243:
        return reg.predict( row['DAYS_BIRTH'] )[0] # need this [0] so it puts value and not a 1-dim array in place
    # fill in any nans while we are at it
    elif np.isnan(row['DAYS_EMPLOYED']):
        return reg.predict( row['DAYS_BIRTH'] )[0] # need this [0] so it puts value and not a 1-dim array in place
    else:
        return row['DAYS_EMPLOYED']
    
application_train_df['DAYS_EMPLOYED'] = application_train_df.apply(fix_DAYS_EMPLOYED, axis = 1)

del reg
del no_weird_vals_df

## replacing NaNs

In [12]:
# replace NaNs in CNT_FAMILY_MEMBERS and make it an int
# need to replace NaNs bc pandas does not support NaNs in ints!

replace_dict = {#'EXT_SOURCE_1':.5,
                #'EXT_SOURCE_3':.5, # there are a lot of missing values, but as it stands it is hard to find anything better to impute with
                'CNT_FAM_MEMBERS':application_train_df['CNT_FAM_MEMBERS'].median(), # not many missing, need an integer so use median
                'AMT_ANNUITY_from_app_train':application_train_df['AMT_ANNUITY_from_app_train'].median(), # not many missing
                'AMT_GOODS_PRICE': application_train_df['AMT_GOODS_PRICE'].median(), # not many missing
                'DAYS_LAST_PHONE_CHANGE': application_train_df['DAYS_LAST_PHONE_CHANGE'].median(), # not many missing
                'OWN_CAR_AGE':15, # eyeballed the car age that has equal TARGET = 0, 1 scores                 
                'APARTMENTS_AVG':0, # A LOT MISSING, having a null here likely means 0 for someone filling out the form        
                'LANDAREA_AVG':0, # A LOT MISSING, having a null here likely means 0 for someone filling out the form                  
                'TOTALAREA_MODE':0, # A LOT MISSING, having a null here likely means 0 for someone filling out the form            
                'OBS_60_CNT_SOCIAL_CIRCLE':0, # not many missing, having a null here likely means 0 for someone filling out the form          
                'AMT_REQ_CREDIT_BUREAU_YEAR':0, # A LOT MISSING, having a null here likely means 0 for someone filling out the form    
                }

application_train_df.fillna(replace_dict, inplace=True)

# convert this to int now that NaNs are gone, which it should be
application_train_df['CNT_FAM_MEMBERS'] = application_train_df['CNT_FAM_MEMBERS'].astype(int)

# this is a final catch all in case other missing values appear in test set
#check_nan = application_train_df.isna().any() 

# replace_dict_EXTRA ={}
# for f in check_nan[check_nan==True].index:
#     replace_dict_EXTRA[f]=application_train_df[f].median()
    
# if replace_dict_EXTRA:
#     print('There were additional replacements!')
#     application_train_df.fillna(replace_dict_EXTRA, inplace=True)

In [13]:
test_1 = application_train_df[application_train_df['EXT_SOURCE_1'].isnull()][['CODE_GENDER','DAYS_BIRTH','EXT_SOURCE_1']].sample(n =5, random_state = 0)

In [14]:
test_2 = application_train_df[application_train_df['EXT_SOURCE_2'].isnull()][['REGION_RATING_CLIENT','DAYS_BIRTH','EXT_SOURCE_2']].sample(n =5, random_state = 3)

In [15]:
test_3 = application_train_df[application_train_df['EXT_SOURCE_3'].isnull()][['DAYS_BIRTH','EXT_SOURCE_3']].sample(n =5, random_state = 0)

In [16]:
# this is a bit slow, but faster than only applying to nan values and then replacing in a loop 
male_df = application_train_df[application_train_df['CODE_GENDER']=='M'][['DAYS_BIRTH','EXT_SOURCE_1']]
male_df = male_df[male_df['EXT_SOURCE_1'].notna()]
m_s, m_i, r_value, p_value, std_err = stats.linregress(male_df)

female_df = application_train_df[application_train_df['CODE_GENDER']=='F'][['DAYS_BIRTH','EXT_SOURCE_1']]
female_df = female_df[female_df['EXT_SOURCE_1'].notna()]
f_s, f_i, r_value, p_value, std_err = stats.linregress(female_df)

nogender_df = application_train_df[['DAYS_BIRTH','EXT_SOURCE_1']]
nogender_df = nogender_df[nogender_df['EXT_SOURCE_1'].notna()]
ng_s, ng_i, r_value, p_value, std_err = stats.linregress(nogender_df)

def EXT1_replace(row):
    if np.isnan(row['EXT_SOURCE_1']) ==False:
        return row['EXT_SOURCE_1']
    else:
        if row['CODE_GENDER']=='M':
            return m_s*row['DAYS_BIRTH'] + m_i
        if row['CODE_GENDER']=='F':
            return f_s*row['DAYS_BIRTH'] + f_i
        if row['CODE_GENDER']=='XNA':
            return ng_s*row['DAYS_BIRTH'] + ng_i
    
application_train_df['EXT_SOURCE_1'] = application_train_df.apply(EXT1_replace, axis = 1)

del male_df
del female_df
del nogender_df
del  r_value, p_value, std_err

In [17]:
# check 
display(test_1)
display(application_train_df.loc[test_1.index.values, ['CODE_GENDER','DAYS_BIRTH','EXT_SOURCE_1']])

,CODE_GENDER,DAYS_BIRTH,EXT_SOURCE_1
SK_ID_CURR,,,
215541,M,-9244,NaN
277842,F,-19019,NaN
196576,M,-12571,NaN
432484,F,-17191,NaN
299977,F,-18594,NaN


,CODE_GENDER,DAYS_BIRTH,EXT_SOURCE_1
SK_ID_CURR,,,
215541,M,-9244,0.255355
277842,F,-19019,0.667924
196576,M,-12571,0.360291
432484,F,-17191,0.612460
299977,F,-18594,0.655029


In [18]:
# this is a bit slow, but faster than only applying to nan values and then replacing in a loop 
r1_df = application_train_df[application_train_df['REGION_RATING_CLIENT']==1][['DAYS_BIRTH','EXT_SOURCE_2']]
r1_df = r1_df[r1_df['EXT_SOURCE_2'].notna()]
r1_s, r1_i, r_value, p_value, std_err = stats.linregress(r1_df)

r2_df = application_train_df[application_train_df['REGION_RATING_CLIENT']==2][['DAYS_BIRTH','EXT_SOURCE_2']]
r2_df = r2_df[r2_df['EXT_SOURCE_2'].notna()]
r2_s, r2_i, r_value, p_value, std_err = stats.linregress(r2_df)

r3_df = application_train_df[application_train_df['REGION_RATING_CLIENT']==3][['DAYS_BIRTH','EXT_SOURCE_2']]
r3_df = r3_df[r3_df['EXT_SOURCE_2'].notna()]
r3_s, r3_i, r_value, p_value, std_err = stats.linregress(r3_df)

def EXT2_replace(row):
    if np.isnan(row['EXT_SOURCE_2']) ==False:
        return row['EXT_SOURCE_2']
    else:
        if row['REGION_RATING_CLIENT']==1:
            return r1_s*row['DAYS_BIRTH'] + r1_i
        if row['REGION_RATING_CLIENT']==2:
            return r2_s*row['DAYS_BIRTH'] + r2_i
        if row['REGION_RATING_CLIENT']==3:
            return r3_s*row['DAYS_BIRTH'] + r3_i
    
application_train_df['EXT_SOURCE_2'] = application_train_df.apply(EXT2_replace, axis = 1)

del r1_df, r2_df, r3_df
del r_value, p_value, std_err

In [19]:
# check
display(test_2)
display(application_train_df.loc[test_2.index.values,['REGION_RATING_CLIENT','DAYS_BIRTH','EXT_SOURCE_2']])

,REGION_RATING_CLIENT,DAYS_BIRTH,EXT_SOURCE_2
SK_ID_CURR,,,
254814,2,-18864,NaN
350771,1,-13779,NaN
285738,2,-24475,NaN
437408,3,-14783,NaN
335830,2,-10766,NaN


,REGION_RATING_CLIENT,DAYS_BIRTH,EXT_SOURCE_2
SK_ID_CURR,,,
254814,2,-18864,0.527566
350771,1,-13779,0.631082
285738,2,-24475,0.548885
437408,3,-14783,0.413344
335830,2,-10766,0.496797


In [20]:
# this is a bit slow, but faster than only applying to nan values and then replacing in a loop 
e3_df = application_train_df[['DAYS_BIRTH','EXT_SOURCE_3']]
e3_df = e3_df[e3_df['EXT_SOURCE_3'].notna()]
e3_s, e3_i, r_value, p_value, std_err = stats.linregress(e3_df)

def EXT3_replace(row):
    if np.isnan(row['EXT_SOURCE_3']) ==False:
        return row['EXT_SOURCE_3']
    else:
        return e3_s*row['DAYS_BIRTH'] + e3_i
    
application_train_df['EXT_SOURCE_3'] = application_train_df.apply(EXT3_replace, axis = 1)

del e3_df
del r_value, p_value, std_err

In [21]:
# check
display(test_3)
display(application_train_df.loc[test_3.index.values,['DAYS_BIRTH','EXT_SOURCE_3']])

,DAYS_BIRTH,EXT_SOURCE_3
SK_ID_CURR,,
339375,-13796,NaN
175526,-14767,NaN
400503,-10197,NaN
450576,-20177,NaN
335597,-14154,NaN


,DAYS_BIRTH,EXT_SOURCE_3
SK_ID_CURR,,
339375,-13796,0.489097
175526,-14767,0.498140
400503,-10197,0.455581
450576,-20177,0.548521
335597,-14154,0.492431


In [22]:
application_train_df.isnull().any()

TARGET                         False
NAME_CONTRACT_TYPE             False
CODE_GENDER                    False
FLAG_OWN_CAR                   False
FLAG_OWN_REALTY                False
CNT_CHILDREN                   False
AMT_INCOME_TOTAL               False
AMT_CREDIT                     False
AMT_ANNUITY_from_app_train     False
AMT_GOODS_PRICE                False
NAME_EDUCATION_TYPE            False
NAME_FAMILY_STATUS             False
REGION_POPULATION_RELATIVE     False
DAYS_BIRTH                     False
DAYS_EMPLOYED                  False
DAYS_REGISTRATION              False
DAYS_ID_PUBLISH                False
OWN_CAR_AGE                    False
FLAG_EMP_PHONE                 False
FLAG_WORK_PHONE                False
FLAG_PHONE                     False
FLAG_EMAIL                     False
CNT_FAM_MEMBERS                False
REGION_RATING_CLIENT           False
WEEKDAY_APPR_PROCESS_START     False
HOUR_APPR_PROCESS_START        False
REG_REGION_NOT_LIVE_REGION     False
R

In [23]:
application_train_df.isnull().values.any()

False

## create new features (log and pwr of continuous features)

In [24]:
# these were explored in plots in a separate notebook

this_feat = 'AMT_INCOME_TOTAL'
application_train_df['log_'+this_feat] = np.log10( application_train_df[this_feat] + 1 ) 

this_feat = 'AMT_CREDIT'
application_train_df['log_'+this_feat] = np.log10( application_train_df[this_feat] + 1 ) 

this_feat = 'AMT_ANNUITY_from_app_train'
application_train_df['pwr_'+this_feat] = (application_train_df[this_feat])**(1/4)

this_feat = 'AMT_GOODS_PRICE'
application_train_df['log_'+this_feat] = np.log10( application_train_df[this_feat] + 1 ) 

this_feat = 'REGION_POPULATION_RELATIVE'
application_train_df['pwr_'+this_feat] = (application_train_df[this_feat])**(1/4)

this_feat = 'DAYS_REGISTRATION' 
application_train_df['pwr_'+this_feat] = (-application_train_df[this_feat])**(1/2)

this_feat = 'OWN_CAR_AGE'
application_train_df['pwr_'+this_feat] = (+application_train_df[this_feat])**(1/2)

this_feat = 'DAYS_LAST_PHONE_CHANGE'
application_train_df['pwr_'+this_feat] = (-application_train_df[this_feat])**(1/2)

this_feat = 'DAYS_EMPLOYED'
application_train_df['pwr_'+this_feat] = (-application_train_df[this_feat])**(1/8)

this_feat = 'DAYS_ID_PUBLISH'
application_train_df['pwr_'+this_feat] = (-application_train_df[this_feat])**(1/1.8)
 
feats_do_drop = ['AMT_INCOME_TOTAL',
                 'AMT_CREDIT',
                 'AMT_ANNUITY_from_app_train',
                 'AMT_GOODS_PRICE',
                 'REGION_POPULATION_RELATIVE',
                 'DAYS_REGISTRATION',
                 'OWN_CAR_AGE',
                 'DAYS_LAST_PHONE_CHANGE',
                 'DAYS_EMPLOYED',
                 'DAYS_ID_PUBLISH',
                  ]

application_train_df.drop(columns=feats_do_drop, inplace=True)

In [25]:
application_train_df.isnull().any()

TARGET                            False
NAME_CONTRACT_TYPE                False
CODE_GENDER                       False
FLAG_OWN_CAR                      False
FLAG_OWN_REALTY                   False
CNT_CHILDREN                      False
NAME_EDUCATION_TYPE               False
NAME_FAMILY_STATUS                False
DAYS_BIRTH                        False
FLAG_EMP_PHONE                    False
FLAG_WORK_PHONE                   False
FLAG_PHONE                        False
FLAG_EMAIL                        False
CNT_FAM_MEMBERS                   False
REGION_RATING_CLIENT              False
WEEKDAY_APPR_PROCESS_START        False
HOUR_APPR_PROCESS_START           False
REG_REGION_NOT_LIVE_REGION        False
REG_REGION_NOT_WORK_REGION        False
LIVE_REGION_NOT_WORK_REGION       False
REG_CITY_NOT_LIVE_CITY            False
REG_CITY_NOT_WORK_CITY            False
LIVE_CITY_NOT_WORK_CITY           False
EXT_SOURCE_1                      False
EXT_SOURCE_2                      False


In [26]:
application_train_df.isnull().values.any()

False

## save as csv

In [27]:
application_train_df.to_csv(save_path + 'application_train_df_cleaned_final.csv', columns = list(application_train_df.columns))

In [28]:
del application_train_df

# bureau wrangling

In [29]:
best_feats = ['SK_ID_CURR',
              'CREDIT_CURRENCY_mode', 
              'CREDIT_ACTIVE_mode', 
              'CREDIT_TYPE_mode',
              'DAYS_CREDIT_min',
              'DAYS_CREDIT_mean',
              'DAYS_ENDDATE_FACT_min',
              'DAYS_CREDIT_median',
              'DAYS_ENDDATE_FACT_median',
            # 'AMT_ANNUITY_from_bureau_mean', # this doesn't have enough non null vals
              'AMT_CREDIT_SUM_DEBT_mean',
              'DAYS_CREDIT_ENDDATE_mean', 
              'DAYS_CREDIT_UPDATE_mean'
                ]

In [30]:
bureau_df_aggregated_final_df = pd.read_csv(agg_data_path + 'bureau_df_aggregated_final.csv', index_col='SK_ID_CURR', usecols=best_feats)

In [31]:
bureau_df_aggregated_final_df.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Data columns (total 11 columns):
DAYS_CREDIT_ENDDATE_mean    261242 non-null float64
DAYS_ENDDATE_FACT_median    230355 non-null float64
DAYS_ENDDATE_FACT_min       230355 non-null float64
AMT_CREDIT_SUM_DEBT_mean    256131 non-null float64
DAYS_CREDIT_mean            263491 non-null float64
DAYS_CREDIT_median          263491 non-null float64
DAYS_CREDIT_min             263491 non-null float64
DAYS_CREDIT_UPDATE_mean     263491 non-null float64
CREDIT_ACTIVE_mode          263491 non-null object
CREDIT_CURRENCY_mode        263491 non-null object
CREDIT_TYPE_mode            263491 non-null object
dtypes: float64(8), object(3)
memory usage: 28.2+ MB


In [32]:
# replace NaNs in object entries for now is pretty safe. 
f_list = list(bureau_df_aggregated_final_df.select_dtypes('object').columns)
bureau_df_aggregated_final_df[f_list] = bureau_df_aggregated_final_df[f_list].fillna(value = 'No/Av')
del f_list

In [33]:
# for categorical feature than have more than one value <1%, condensed the <1% values and define a new condensed feature

for feat in bureau_df_aggregated_final_df.select_dtypes('object').columns:
    percent_srs = bureau_df_aggregated_final_df[feat].value_counts() /len(bureau_df_aggregated_final_df[feat])
    percent_less_than_1 = list(percent_srs[percent_srs <.01].index)
    if len(percent_less_than_1)>1:
        bureau_df_aggregated_final_df[feat + '_condensed'] = bureau_df_aggregated_final_df[feat].apply(lambda x: '<1%' if x in percent_less_than_1 else x)
        bureau_df_aggregated_final_df.drop(columns=[feat], inplace=True)
      
del feat
del percent_srs
del percent_less_than_1

In [34]:
def log_modulus_transformation(x):
    return np.sign(x)*np.log10( np.abs(x)+1)

def pwr_transformation(x,pwr):
    return np.sign(x)*( np.abs(x) )**pwr

In [35]:
# TRANSFORMATIONS, then drop original features. 

bureau_df_aggregated_final_df['pwr_DAYS_CREDIT_mean'] = pwr_transformation(bureau_df_aggregated_final_df['DAYS_CREDIT_mean'], 1/1.7)
bureau_df_aggregated_final_df['pwr_DAYS_ENDDATE_FACT_min'] = pwr_transformation(bureau_df_aggregated_final_df['DAYS_ENDDATE_FACT_min'],1/1.3)
bureau_df_aggregated_final_df['pwr_DAYS_CREDIT_median'] = pwr_transformation(bureau_df_aggregated_final_df['DAYS_CREDIT_median'],1/2)
bureau_df_aggregated_final_df['pwr_DAYS_ENDDATE_FACT_median'] = pwr_transformation(bureau_df_aggregated_final_df['DAYS_ENDDATE_FACT_median'],1/2)
bureau_df_aggregated_final_df['LogMod_AMT_CREDIT_SUM_DEBT_mean'] = log_modulus_transformation(bureau_df_aggregated_final_df['AMT_CREDIT_SUM_DEBT_mean'])
bureau_df_aggregated_final_df['pwr_DAYS_CREDIT_ENDDATE_mean'] = pwr_transformation(bureau_df_aggregated_final_df['DAYS_CREDIT_ENDDATE_mean'], .65)
bureau_df_aggregated_final_df['pwr_DAYS_CREDIT_UPDATE_mean'] = pwr_transformation(bureau_df_aggregated_final_df['DAYS_CREDIT_UPDATE_mean'],.2)

bureau_df_aggregated_final_df.drop(columns=['DAYS_CREDIT_mean',
                                            'DAYS_ENDDATE_FACT_min',                                            
                                            'DAYS_CREDIT_median',
                                            'DAYS_ENDDATE_FACT_median',   
                                            'AMT_CREDIT_SUM_DEBT_mean',
                                            'DAYS_CREDIT_ENDDATE_mean',
                                            'DAYS_CREDIT_UPDATE_mean',
                                            ], inplace=True)

In [36]:
replace_dict = {'DAYS_CREDIT_min':-1262,
                'pwr_DAYS_CREDIT_mean':54,
                'pwr_DAYS_ENDDATE_FACT_min':134,
                'pwr_DAYS_CREDIT_median':26,
                'pwr_DAYS_ENDDATE_FACT_median':22,
                'LogMod_AMT_CREDIT_SUM_DEBT_mean':bureau_df_aggregated_final_df['LogMod_AMT_CREDIT_SUM_DEBT_mean'].median(),
                'pwr_DAYS_CREDIT_ENDDATE_mean':0,
                'pwr_DAYS_CREDIT_UPDATE_mean':-3.23,    
                }

bureau_df_aggregated_final_df.fillna(replace_dict, inplace=True)

In [37]:
bureau_df_aggregated_final_df.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Data columns (total 11 columns):
DAYS_CREDIT_min                    307511 non-null float64
CREDIT_ACTIVE_mode                 307511 non-null object
CREDIT_CURRENCY_mode_condensed     307511 non-null object
CREDIT_TYPE_mode_condensed         307511 non-null object
pwr_DAYS_CREDIT_mean               307511 non-null float64
pwr_DAYS_ENDDATE_FACT_min          307511 non-null float64
pwr_DAYS_CREDIT_median             307511 non-null float64
pwr_DAYS_ENDDATE_FACT_median       307511 non-null float64
LogMod_AMT_CREDIT_SUM_DEBT_mean    307511 non-null float64
pwr_DAYS_CREDIT_ENDDATE_mean       307511 non-null float64
pwr_DAYS_CREDIT_UPDATE_mean        307511 non-null float64
dtypes: float64(8), object(3)
memory usage: 28.2+ MB


In [38]:
bureau_df_aggregated_final_df.to_csv(save_path + 'bureau_df_aggregated_wrangled_final.csv', columns = list(bureau_df_aggregated_final_df.columns))

In [39]:
del bureau_df_aggregated_final_df

# installments payments wrangling

In [40]:
best_feats = ['SK_ID_CURR',
              'NUM_OF_LOANS',
              'DAYS_PAYMENT_LATE_median',
              'DAYS_PAYMENT_LATE_sd',
              'NUM_TIMES_LATE',
              'NUM_TIMES_EARLY',
              'AMT_OVERPAY_MEAN',
              'AMT_OVERPAY_SD',
              'NUM_TIMES_OVERPAY',
              'TERMS_CHANGE_TIMES',
              'NUM_INSTALMENT_VERSION_max',
              'NUM_INSTALMENT_VERSION_min',
              'DAYS_INSTALMENT_mean',
              'DAYS_INSTALMENT_min',
              'DAYS_ENTRY_PAYMENT_mean',
              'DAYS_ENTRY_PAYMENT_min',
              'AMT_INSTALMENT_mean',
              'AMT_INSTALMENT_max',
              'AMT_INSTALMENT_min',
              'AMT_PAYMENT_mean',
              'AMT_PAYMENT_max',
              'AMT_PAYMENT_min',
              'NUM_INSTALMENT_NUMBER_mean',
              'NUM_INSTALMENT_NUMBER_max',
             ]            
              
#               'NUM_INSTALMENT_VERSION_min',
#               'DAYS_INSTALMENT_mean',
#               'DAYS_INSTALMENT_min',
#               'DAYS_ENTRY_PAYMENT_median',
#               'DAYS_ENTRY_PAYMENT_min',
#               'AMT_INSTALMENT_mean',
#               'AMT_INSTALMENT_max',
#               'AMT_INSTALMENT_min',
#               'AMT_PAYMENT_mean',
#               'AMT_PAYMENT_max',
#               'AMT_PAYMENT_min',
#               'NUM_INSTALMENT_NUMBER_mean',
#             ]

In [41]:
installments_payments_df_agg_final = pd.read_csv(agg_data_path + 'installments_payments_df_final.csv', index_col='SK_ID_CURR', usecols=best_feats)

In [42]:
installments_payments_df_agg_final.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Data columns (total 23 columns):
NUM_OF_LOANS                  291643 non-null float64
DAYS_PAYMENT_LATE_median      291635 non-null float64
DAYS_PAYMENT_LATE_sd          290947 non-null float64
NUM_TIMES_LATE                291643 non-null float64
NUM_TIMES_EARLY               291643 non-null float64
AMT_OVERPAY_MEAN              290913 non-null float64
AMT_OVERPAY_SD                290229 non-null float64
NUM_TIMES_OVERPAY             291643 non-null float64
TERMS_CHANGE_TIMES            291643 non-null float64
NUM_INSTALMENT_VERSION_max    291643 non-null float64
NUM_INSTALMENT_VERSION_min    291643 non-null float64
DAYS_INSTALMENT_mean          291643 non-null float64
DAYS_INSTALMENT_min           291643 non-null float64
DAYS_ENTRY_PAYMENT_mean       291635 non-null float64
DAYS_ENTRY_PAYMENT_min        291635 non-null float64
AMT_INSTALMENT_mean           291643 non-null float64
AMT_INSTALMENT_max  

In [43]:
def log_modulus_transformation(x):
    return np.sign(x)*np.log10( np.abs(x)+1)

def pwr_transformation(x,pwr):
    return np.sign(x)*( np.abs(x) )**pwr

def transform_drop_replace(df_in, old_feat, trans,  replace_w, exp=0):
    
    if trans=='pwr':
        new_feat = 'pwr_'+ old_feat
        df_in[new_feat] = pwr_transformation(df_in[old_feat],exp) 
        df_in.drop(columns = [old_feat], inplace = True)
        
        if type(replace_w)==int:
            df_in.fillna({new_feat:replace_w}, inplace = True)
        elif replace_w == 'median':
            df_in.fillna({new_feat:df_in[new_feat].median()}, inplace = True)
        elif replace_w == 'mean':
            df_in.fillna({new_feat:df_in[new_feat].mean()}, inplace = True)
        
    if trans=='logmod':
        new_feat = 'LogMod_'+ old_feat
        df_in[new_feat] = log_modulus_transformation(df_in[old_feat]) 
        df_in.drop(columns = [old_feat], inplace = True)
        
        if type(replace_w)==int:
            df_in.fillna({new_feat:replace_w}, inplace = True)
        elif replace_w == 'median':
            df_in.fillna({new_feat:df_in[new_feat].median()}, inplace = True)
        elif replace_w == 'mean':
            df_in.fillna({new_feat:df_in[new_feat].mean()}, inplace = True)
        

In [44]:
# TRANSFORMATIONS, then drop original features. 


transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'DAYS_PAYMENT_LATE_median', 
                       trans = 'pwr', 
                       replace_w = 'median',
                       exp = 1/1.7, 
                      )

transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat ='DAYS_PAYMENT_LATE_sd', 
                       trans = 'logmod',  
                       replace_w = 'median',
                       )

transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat ='NUM_TIMES_LATE', 
                       trans = 'pwr', 
                       exp = 1/1.5, 
                       replace_w = 'median')

transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat ='NUM_TIMES_EARLY', 
                       trans = 'logmod', 
                       replace_w ='median')


transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'TERMS_CHANGE_TIMES', 
                       trans = 'pwr', 
                       exp = 1/2, 
                       replace_w ='median')

transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'DAYS_INSTALMENT_mean', 
                       trans = 'pwr', 
                       exp = 1/3.5, 
                       replace_w ='median')

transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat ='DAYS_ENTRY_PAYMENT_mean', 
                       trans = 'pwr', 
                       exp = 1/3, 
                       replace_w ='median')
                       
transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'AMT_INSTALMENT_mean', 
                       trans = 'logmod', 
                       replace_w = 'median')       
                       
transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'AMT_INSTALMENT_max', 
                       trans = 'pwr', 
                       exp = 1/6, 
                       replace_w ='median')
                       
transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'AMT_INSTALMENT_min', 
                       trans = 'pwr', 
                       exp = 1/4, 
                       replace_w ='median')                       
                       
transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'AMT_PAYMENT_mean', 
                       trans = 'logmod', 
                       replace_w = 'median')                       

transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'AMT_PAYMENT_max', 
                       trans = 'logmod', 
                       replace_w = 'median')                       
                       
transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'AMT_PAYMENT_min', 
                       trans = 'pwr', 
                       exp =1/5.2, 
                       replace_w = 'median')                       
                       
transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'NUM_INSTALMENT_NUMBER_mean', 
                       trans = 'logmod', 
                       replace_w = 'median')                       
                       
transform_drop_replace(df_in = installments_payments_df_agg_final, 
                       old_feat = 'NUM_INSTALMENT_NUMBER_max', 
                       trans = 'logmod', 
                       replace_w = 'median')

In [45]:
remaining_replace_list=['NUM_OF_LOANS',
                        'AMT_OVERPAY_MEAN',
                        'AMT_OVERPAY_SD',
                        'NUM_TIMES_OVERPAY',
                        'NUM_INSTALMENT_VERSION_max',
                        'NUM_INSTALMENT_VERSION_min',
                        'DAYS_INSTALMENT_min',
                        'DAYS_ENTRY_PAYMENT_min', 
                       ]
    
for f in remaining_replace_list:
    installments_payments_df_agg_final.fillna({f:installments_payments_df_agg_final[f].median()}, inplace=True)

In [46]:
installments_payments_df_agg_final.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Data columns (total 23 columns):
NUM_OF_LOANS                         307511 non-null float64
AMT_OVERPAY_MEAN                     307511 non-null float64
AMT_OVERPAY_SD                       307511 non-null float64
NUM_TIMES_OVERPAY                    307511 non-null float64
NUM_INSTALMENT_VERSION_max           307511 non-null float64
NUM_INSTALMENT_VERSION_min           307511 non-null float64
DAYS_INSTALMENT_min                  307511 non-null float64
DAYS_ENTRY_PAYMENT_min               307511 non-null float64
pwr_DAYS_PAYMENT_LATE_median         307511 non-null float64
LogMod_DAYS_PAYMENT_LATE_sd          307511 non-null float64
pwr_NUM_TIMES_LATE                   307511 non-null float64
LogMod_NUM_TIMES_EARLY               307511 non-null float64
pwr_TERMS_CHANGE_TIMES               307511 non-null float64
pwr_DAYS_INSTALMENT_mean             307511 non-null float64
pwr_DAYS_ENTRY_PAYMENT_mean   

In [47]:
installments_payments_df_agg_final.to_csv(save_path + 'installments_payments_wrangled_df_final.csv', columns = list(installments_payments_df_agg_final.columns))

In [48]:
del installments_payments_df_agg_final

# create final single wrangled CSV

In [49]:
save_path

'Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/wrangling/TRAINING_DATA_create_final_wrangled_csv/'

In [50]:
df_to_merge_0 = pd.read_csv(save_path + 'application_train_df_cleaned_final.csv', index_col = 'SK_ID_CURR')
df_to_merge_1 = pd.read_csv(save_path + 'bureau_df_aggregated_wrangled_final.csv', index_col = 'SK_ID_CURR')
df_to_merge_2 = pd.read_csv(save_path + 'installments_payments_wrangled_df_final.csv', index_col = 'SK_ID_CURR')

dfs = (df_to_merge_0,
       df_to_merge_1,
       df_to_merge_2,
       )

# these all have the same index (SK_ID_CURR) so no risk for funny business
total_df = reduce( lambda left, right: pd.merge( left, right, left_index = True, right_index = True), dfs)
# from https://stackoverflow.com/questions/23668427/pandas-joining-multiple-dataframes-on-columns

del dfs
del df_to_merge_0
del df_to_merge_1
del df_to_merge_2

In [51]:

total_df.to_csv(save_path + 'complete_initial_wrangled_data.csv', columns = list(total_df.columns))

In [52]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Data columns (total 86 columns):
TARGET                               307511 non-null int64
NAME_CONTRACT_TYPE                   307511 non-null object
CODE_GENDER                          307511 non-null object
FLAG_OWN_CAR                         307511 non-null object
FLAG_OWN_REALTY                      307511 non-null object
CNT_CHILDREN                         307511 non-null int64
NAME_EDUCATION_TYPE                  307511 non-null object
NAME_FAMILY_STATUS                   307511 non-null object
DAYS_BIRTH                           307511 non-null int64
FLAG_EMP_PHONE                       307511 non-null int64
FLAG_WORK_PHONE                      307511 non-null int64
FLAG_PHONE                           307511 non-null int64
FLAG_EMAIL                           307511 non-null int64
CNT_FAM_MEMBERS                      307511 non-null int64
REGION_RATING_CLIENT                 307511 non-null